3_21 모델학습 후 예측한 결과가 심통치않아 새로 전처리해보기로 하였다.

In [15]:
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

In [16]:
#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file('test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

In [17]:
#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')

In [18]:
train_text[:100]

'id\tdocument\tlabel\n9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0\n3819312\t흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나\t1\n10265843'

In [19]:
a= train_text.split('\n')[1:][0]

a.split('\t')[2]

'0'

In [20]:
# X 값 추출 = 리뷰문장 == 피쳐
x_train = [ row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0]
x_train[:3]

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다']

In [21]:
#y값 추출  == target
y_train= np.array([ [int(row.split('\t')[2])] for row in train_text.split('\n')[1:] if row.count('\t')>0])
y_test= np.array([ [int(row.split('\t')[2])] for row in test_text.split('\n')[1:] if row.count('\t')>0])
print(y_train.shape) # (150000개의 평점 , 1개의 열)
y_train

(150000, 1)


array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [22]:
from konlpy.tag import Okt  #트위터에서 만든 한국어분석 메서드
from konlpy.tag import Kkma  #서울대에서 만든거 morphs== 형태소 분석


x_tokenized = [ ]
for i in x_train[:100]:
    x_tokenized.append( Okt().morphs( i ) ) 
    
x_tokenized[:5]

[['아', '더빙', '..', '진짜', '짜증나네요', '목소리'],
 ['흠',
  '...',
  '포스터',
  '보고',
  '초딩',
  '영화',
  '줄',
  '....',
  '오버',
  '연기',
  '조차',
  '가볍지',
  '않구나'],
 ['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다'],
 ['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정'],
 ['사이',
  '몬페',
  '그',
  '의',
  '익살스런',
  '연기',
  '가',
  '돋보였던',
  '영화',
  '!',
  '스파이더맨',
  '에서',
  '늙어',
  '보이기만',
  '했던',
  '커스틴',
  '던스트',
  '가',
  '너무나도',
  '이뻐',
  '보였다']]

. ! ? ;;; 은 는 이 가 => 불용어 제거
불용어 빈도수가 늘어남 = 중요도 판별하는 순위가 되기 때문에
=> TF-IDF  TF= 빈도수  IDF = 역 빈도수 

0 부정문들만 모여있는 문서
1 긍정문들만 모여있는 문서

=> TF-IDF 알고리즘은 두 문서에 겹치는 단어들 동시에 많이 쓰이는 단어들은 점수를 낮게
=> 겹치지않는 단어들은 점수를 높게 매겨준다.

In [23]:
import pandas as pd

x_train[:100] #1차원


y_train[:100]  #2차원 

x_df = pd.DataFrame(x_train[:1000])
# print(x_df)

y_df = pd.DataFrame(y_train[:1000])
# print(y_df)

# pd.DataFrame(x_df, y_df, columns=['x','y'], axis= 1)

df = pd.concat([x_df, y_df], axis=1)

# df
# df.rename(columns={0, 0 : 'x','y'})
# df_1= df[ (df.iloc[1:] ==0) ] 


bad = df[df.iloc[:,1]==0]
good = df[df.iloc[:,1]==1]

bad
# TF-IDF 단어 중요도 점수를 매긴다.

,0,0
0,아 더빙.. 진짜 짜증나네요 목소리,0
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
5,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,원작의 긴장감을 제대로 살려내지못했다.,0
...,...,...
993,마이클베이 최근작을 볼때마다 느낀다. 블록버스터 액션장르라도 탄탄한 시나리오는 영화...,0
994,0점은없나?1점은 너무 후하네,0
995,미달이는.. 연예계를 바라보는 태도를 고쳐야 한다.,0
996,파괴된 관객들.,0


In [82]:
# df = df.rename(columns={0: 'A', '0': 'B'})

df.columns.values[0] = 'text'
df.columns.values[1] = 'label'

,text,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
995,미달이는.. 연예계를 바라보는 태도를 고쳐야 한다.,0
996,파괴된 관객들.,0
997,알콜중독자들 덕에 평점이 낮은가?,1
998,중견배우들이 만들어놓은 긴장감이 나름 긴장감있게 싸워보려했던 도술사들에 의해 헛웃음...,0


In [89]:
df['text']

KeyError: 'text'

In [65]:
good.iloc[:,0]

goodlist= np.array( [good.iloc[:,0]] )
# print(goodlist)
badlist= np.array( [ bad.iloc[:,0]])
# df.iloc[:,0]

gl=[ ]
for i in goodlist[0]:
    gl.append(i)
bl= [ ]
for i in badlist[0]:
    bl.append(i)
    
texts= [gl,bl]

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

vectorizer = TfidfVectorizer( tokenizer= Okt().morphs ) # 매개변수를 조정하여 다양한 옵션을 적용할 수 있습니다.
X = vectorizer.fit_transform(texts)
# y = df['label']

# X=test111.iloc[:,0]
# y=test111.iloc[:,1]

# X

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
# 딥러닝 모델
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
# from tensorflow.keras.

#딥러닝기반 언어(텍스트정제 => 숫자로된 시계열데이터로 들어온다 )처리모델 

model = Sequential()

#들어온 시퀀스데이터를 벡터로 변환한다.5000==단어사전크기, 600 차원은 클수록 좋다. 인풋랭스는 시퀀스데이터의 길이이다.
model.add( Embedding( 3000, 200, input_length = 3468)) #입력층

model.add( LSTM( units = 50) )   #은닉층,  units: 뉴런갯수 많을수록 좋다.

#0부정,1긍정
model.add( Dense( 2, activation='softmax' ))  #출력층
#activation='softmax'이면 자동으로 loss='sparse_categorical_crossentropy'이다.


# compile: ex) c언어 소스코드 => 컴터가 못알아들음 => 그럼면 알아들을수있게 컴파일해준다.
#컴파일  , optimazer, loss ,metrics
# sparse_categorical_crossentropy는 categorical_crossentropy와 동일하지만, 정수 형태의 레이블을 사용
model.compile(optimizer='adam',  loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
y_train=np.array([
    [0],
    [1]
])

X_tfidf.toarray()

array([[0.01199996, 0.00654543, 0.00436362, ..., 0.00109091, 0.        ,
        0.00327272],
       [0.05326131, 0.02441143, 0.00887688, ..., 0.00110961, 0.00155952,
        0.00443844]])

In [ ]:
model.fit( X_tfidf.toarray(), y_train, epochs=5, batch_size = 500 , validation_split = 0.2 )

Epoch 1/5
1/1 [==============================] - 9s 9s/step - loss: 0.6985 - acc: 0.0000e+00 - val_loss: 0.7525 - val_acc: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 3s 3s/step - loss: 0.6371 - acc: 1.0000 - val_loss: 0.8222 - val_acc: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 3s 3s/step - loss: 0.5789 - acc: 1.0000 - val_loss: 0.8994 - val_acc: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 3s 3s/step - loss: 0.5222 - acc: 1.0000 - val_loss: 0.9888 - val_acc: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 3s 3s/step - loss: 0.4653 - acc: 1.0000 - val_loss: 1.0960 - val_acc: 0.0000e+00


In [ ]:
#테스트 문자열 생성
# test = ['프로젝트 개망했다.'] 
test = ['상당히 볼만했다. 재미있었다.']

test = vectorizer.transform(test)
test.toarray()

test = idf.transform( test )
test.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
model.predict(test.toarray())

1/1 [==============================] - 1s 1s/step


array([[0.6658059 , 0.33419412]], dtype=float32)